In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
url = 'http://www.nhl.com/scores/htmlreports/20182019/PL020658.HTM'
response = requests.get(url)

# lxml parser required
soup = BeautifulSoup(response.text)

In [4]:
trs = soup.find_all('tr', class_='evenColor')

In [5]:
play_by_play_data = {
    'event_id' : [],
    'period' : [],
    'strength' : [],
    'time_elapsed' : [],
    'event_type' : [],
    'event' : [],
    'visitor_on_ice' : [],
    'home_on_ice' : []
}

key_lookup = {
    1 : 'event_id',
    3 : 'period',
    5 : 'strength',
    7 : 'time_elapsed',
    9 : 'event_type',
    11 : 'event',
    13 : 'visitor_on_ice',
    15 : 'home_on_ice'
}
pattern = r'\n+'

for row in trs[4:]:
    for i, cell in enumerate(row):
        
        if i in [1, 3, 5, 7, 9, 11, 13, 15]:
            text = cell.text.replace('\xa0', ' ')
            text = re.sub(pattern, '', text)
            
            key = key_lookup.get(i)
            play_by_play_data[key].append(text)

In [6]:
df = pd.DataFrame(play_by_play_data)

In [28]:
home_on_ice = ( df['home_on_ice']
                .str.strip()
                .str.replace('[aA-zZ]', '')
                .str.split(' ', expand=True) )
home_cols = ['home_1', 'home_2', 'home_3', 'home_4', 'home_5', 'home_6']
home_on_ice.columns = home_cols

visitor_on_ice = ( df['visitor_on_ice']
                   .str.strip()
                   .str.replace('[aA-zZ]', '')
                   .str.split(' ', expand=True) )
visitor_cols = ['visitor_1', 'visitor_2', 'visitor_3', 'visitor_4', 'visitor_5', 'visitor_6']
visitor_on_ice.columns = visitor_cols

In [31]:
df[home_cols] = home_on_ice
df[visitor_cols] = visitor_on_ice

In [34]:
df.head()

,event_id,period,strength,time_elapsed,event_type,event,visitor_on_ice,home_on_ice,home_1,home_2,home_3,home_4,home_5,home_6,visitor_1,visitor_2,visitor_3,visitor_4,visitor_5,visitor_6
0,5,1,EV,0:0020:00,FAC,N.J won Neu. Zone - N.J #19 ZAJAC vs BUF #22 L...,19C 20C 44L 6D 28D 1G,21R 22L 49L 6D 19D 40G,21,22,49,6,19,40,19,20,44,6,28,1
1,6,1,,0:0719:53,STOP,OFFSIDE,19C 20C 44L 6D 28D 1G,21R 22L 49L 6D 19D 40G,21,22,49,6,19,40,19,20,44,6,28,1
2,7,1,EV,0:0719:53,FAC,BUF won Neu. Zone - N.J #37 ZACHA vs BUF #22 L...,37C 23R 56L 6D 28D 1G,21R 22L 49L 6D 19D 40G,21,22,49,6,19,40,37,23,56,6,28,1
3,8,1,EV,0:1219:48,SHOT,"BUF ONGOAL - #6 SCANDELLA, Snap, Off. Zone, 65...",37C 23R 56L 6D 28D 1G,21R 22L 49L 6D 19D 40G,21,22,49,6,19,40,37,23,56,6,28,1
4,9,1,,0:1319:47,STOP,GOALIE STOPPED,37C 23R 56L 6D 28D 1G,21R 22L 49L 6D 19D 40G,21,22,49,6,19,40,37,23,56,6,28,1
